In [44]:
import pandas as pd

#Giving the file path to variable

file = pd.ExcelFile('/content/Timetable-fall-24.xlsx')

#A list containing all data frames

all_DFs = []

#Keywords for matching of labs

lab_keywords = ['C-Margala 1', 'C-Margala 3', 'C-Margala 4', 'C-Rawal 1', 'Rawal 3 (B-232)', 'C-Rawal 4', 'C-GPU Lab', 'A-Karakoram 1', 'A-Karakoram 2', 'A-Karakoram 3', 'A-Mehran 1', 'A-Mehran 2', 'B-Digital', 'A-CALL-1', 'A-CALL-2', 'A-CALL-3']

#For loop in order to look into all sheets

for sheet in file.sheet_names:

    #Read sheets one by one

    df = pd.read_excel(file,sheet,header=4)

    #Drop columns containing all nulls

    df = df.dropna(axis=1,how='all')

    #List containing data

    data = []

    #Prev time for assigning prev time to index is current time is empty

    prev_time = None

    #Variable for storing lab time

    lab_time = None

    #Bool for checking if sheet is thursday as only Labs are present on Thursday in Excel file

    is_Thursday = (sheet == "Thursday")

    #Iterating through each index in the excel sheet

    for i, row in enumerate(df.index):
        for j,col in enumerate(df.columns):

            #Assigning the index to cell

            cell = df.at[row,col]

            #For time if it is Unnamed then assign the prev col time to it else assign the 1st row for respective column to it

            if 'Unnamed' in col:
                time = prev_time
            else:
                time = df.columns[j]
                prev_time = time

            start_index = str(cell).find('(')
            end_index = str(cell).find(')')
            extracted_string = str(cell)[start_index + 1:end_index]

            #Creation of new row as Class = 1st col of respected row Day = SheetName and Course = Str(index)

            new_row = {
                'Class' : df.at[i, df.columns[0]],
                'Day' : sheet,
                'Course' : str(cell).split('(')[0],
            }

            if start_index != -1 and end_index != -1:
                new_row['Section'] = extracted_string
            else:
                new_row['Section'] = new_row['Course']

            #Now checking for time and type

            #If it is thursday and Class is present in lab-keywords then Type = Lab and time would be taken from 39th row

            if is_Thursday:
                if new_row['Class'] in lab_keywords:
                    new_row['Type'] = 'Lab'
                    lab_time = df.iat[38, j] if j < len(df.columns) else ''
                    new_row['Time'] = lab_time

                #Else it would be type = theory and time = time

                else:
                    new_row['Type'] = 'Theory'
                    new_row['Time'] = time

            #On other days It'd always be type = theory and time = time

            else:
                new_row['Type'] = 'Theory'
                new_row['Time'] = time

            #Append the new rows into the data

            data.append(new_row)

    #Making the data frame of the data

    new_data = pd.DataFrame(data)

    #Remove all null rows

    new_data.dropna(inplace = True)

    #Remove all rows containing ''

    new_data = new_data[~(new_data == '').any(axis = 1)]

    #Remove all rows containing nan

    new_data = new_data[~(new_data == 'nan').any(axis = 1)]

    #Remove all rows containing time = room

    new_data = new_data[~(new_data['Time'] == 'Room')]

    #Remove all rows containing time = lab

    new_data = new_data[~(new_data['Time'] == 'Lab')]

    #Remove all rows containing class = lab

    new_data = new_data[~(new_data['Class'] == 'Lab')]

    #Reset the indexes in data frams

    new_data.reset_index(drop = True, inplace = True)

    #Append data frame to ALL DFS list and continue the loop till end of lists

    all_DFs.append(new_data)


#Now displaying and saving the dafa frames into csv file

for idx, df in enumerate(all_DFs):
    csv_filename = f'{file.sheet_names[idx]}.csv'
    df.to_csv(csv_filename, index=False)

In [45]:
import pandas as pd
import os
import random
from IPython.display import HTML, display

# Function to load CSV files from a specific directory
def load_csv_files(directory='/content/'):
    file_list = [f for f in os.listdir(directory) if f.endswith('.csv')]
    dfs = []
    for file in file_list:
        print(f"Loading file: {file}")
        df = pd.read_csv(os.path.join(directory, file))
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Function to generate a random color
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Function to get a color for each course, with random colors
def get_color(course_code, color_map):
    if course_code not in color_map:
        color_map[course_code] = random_color()
    return color_map[course_code]

# Function to filter and display course details with colors
def display_course_details(courses, departments, df):
    color_map = {}
    filtered_data = df[
        (df['Course'].str.strip().isin(courses)) &
        (df['Section'].str[:2].isin(departments))
    ]

    # Check if there's any data for the given course codes and departments
    if filtered_data.empty:
        print(f"No details found for the given courses in departments {departments}")
        return

    # Create an HTML table with colored rows
    html_output = "<html><body><h2>Details for the courses and departments:</h2><table border='1' style='border-collapse: collapse;'>"
    html_output += "<tr><th>Class</th><th>Day</th><th>Course</th><th>Section</th><th>Type</th><th>Time</th></tr>"

    for _, row in filtered_data.iterrows():
        color = get_color(row['Course'].strip(), color_map)
        html_output += f"<tr style='background-color: {color};'><td>{row['Class']}</td><td>{row['Day']}</td><td>{row['Course']}</td><td>{row['Section']}</td><td>{row['Type']}</td><td>{row['Time']}</td></tr>"

    html_output += "</table></body></html>"

    # Display the HTML output
    display(HTML(html_output))

# Load CSV files from the /content/ directory
df = load_csv_files()


# Get user input for number of courses, course codes, number of departments, and department codes
num_courses = int(input("Enter the number of courses you want to filter by: ").strip())
courses = [input(f"Enter course Name(e.g PF) {i+1}: ").strip() for i in range(num_courses)]

num_departments = int(input("Enter the number of departments you want to filter by: ").strip())
departments = [input(f"Enter department (e.g DS) {i+1}: ").strip() for i in range(num_departments)]

# Display the filtered details with random colors
display_course_details(courses, departments, df)


Loading file: Friday.csv
Loading file: Monday.csv
Loading file: Wednesday.csv
Loading file: Tuesday.csv
Loading file: Thursday.csv
Enter the number of courses you want to filter by: 5
Enter course Name(e.g PF) 1: Data Ware & BI
Enter course Name(e.g PF) 2: OS
Enter course Name(e.g PF) 3: Info Sec
Enter course Name(e.g PF) 4: Comp Net
Enter course Name(e.g PF) 5: Deep Learn for Perc
Enter the number of departments you want to filter by: 1
Enter department (e.g DS) 1: DS


Class,Day,Course,Section,Type,Time
C-403,Monday,OS,DS-B,Theory,03:55-05:15
C-404,Monday,OS,DS-A,Theory,01:00-02:20
C-405,Monday,Data Ware & BI,DS-D,Theory,02:30-03:50
C-406,Monday,Comp Net,DS-M,Theory,02:30-03:50
C-406,Monday,Info Sec,DS-M,Theory,03:55-05:15
C-407,Monday,Info Sec,DS-N,Theory,02:30-03:50
C-407,Monday,Comp Net,DS-N,Theory,03:55-05:15
A-108,Monday,Data Ware & BI,DS-C,Theory,11:30-12:50
A-Karakoram 3,Monday,Comp Net,DS-U,Theory,02:30-03:50
C-308,Wednesday,Comp Net,DS-M,Theory,02:30-03:50
